In [1]:
import os
import json
import random

root='/media/jh/新加卷/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/'
all_core='all-subjects'
def process_data(root,dev,en_de,datas):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'subject' in d:
                tmp=d
                break
        item={}
        item['title']=tmp['title']
        item['abstract']=tmp['abstract']
        item['dcterms:subject']=tmp['dcterms:subject']
        item['subject']=tmp['subject']
        datas.append(item)
    for dirs in os.listdir(root+f'{all_core}/data/{dev}'):
        for file in os.listdir(root+f'{all_core}/data/{dev}/{dirs}/{en_de}'):
            get_train_data(root+f'{all_core}/data/{dev}/{dirs}/{en_de}/{file}')
     
train_datas = []
en_de='en'
dev='train'
process_data(root,dev,en_de,train_datas)
en_de='de'
dev='train'
process_data(root,dev,en_de,train_datas)
random.shuffle(train_datas)

dev_datas=[]
en_de='en'
dev='dev'
process_data(root,dev,en_de,dev_datas)
en_de='de'
dev='dev'
process_data(root,dev,en_de,dev_datas)
random.shuffle(dev_datas)

import json
root='/media/jh/新加卷'
file_path=f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())

print(data[0])
gnd_id2name={}
for i in data:
    gnd_id2name[i['Code']]=i['Name']
    for j in i['Alternate Name']:
        gnd_id2name[i['Code']]=gnd_id2name[i['Code']]+','+j
print(i['Code'],gnd_id2name[i['Code']])


import json

print('len(train_datas)=',len(train_datas))
print('len(dev_datas)=',len(dev_datas))
datas=train_datas
print('len(datas)=',len(datas))
# datas数据样本随机打乱
import random
print(datas[0])
random.shuffle(datas)
print(datas[0])
x_train=[]
s1s=[]
cnt=0
gnd_set=set()
for t in datas:
    if type(t['title'])!=str:
        t['title']=' '.join(t['title'])
    if type(t['abstract'])!=str:
        t['abstract']=' '.join(t['abstract'])
    s1s.append(t['title']+'.'+t['abstract'])
    for gnd in t['dcterms:subject']:
        if type(gnd)==dict:
            gnd_id=gnd['@id']
        else:
            continue
        cnt+=1
        if gnd_id in gnd_id2name:
            gnd_set.add(gnd_id)
            gnd_val=gnd_id2name[gnd_id]
            item={}
            item['sentence2']= 'Name is '+gnd_val
            item['sentence1']=t['title']+'.'+t['abstract']
            item['label']=1
            x_train.append(item)
print(len(x_train))      
print(cnt) 
print(len(s1s)) #输入句子有
print(len(gnd_id2name.values()))   
print(len(gnd_set)) #gnd_id有 25371个

{'Code': 'gnd:4003694-7', 'Classification Number': '00', 'Classification Name': 'Unspezifische Allgemeinwörter', 'Name': 'Ausbreitung', 'Alternate Name': [], 'Related Subjects': [], 'Source': 'Du.'}
gnd:1163295434 Airfix Marke
len(train_datas)= 81937
len(dev_datas)= 13666
len(datas)= 81937
{'title': 'Empowerment in der sozialen Arbeit : eine Einführung', 'abstract': ['Das Empowerment-Konzept formuliert eine deutliche Abkehr vom Defizit-Blickwinkel auf die Adressaten Sozialer Arbeit. Handlungsziel der Empowerment-Praxis ist es, die vorhandenen Fähigkeiten der Adressaten sozialer Dienstleistungen zu autonomer Alltagsregie und Lebensorganisation zu kräftigen und Ressourcen freizusetzen, mit deren Hilfe sie die eigenen Lebenswege selbstbestimmt gestalten können. Das Buch bietet eine leicht verständliche Einführung in Theorie und Praxis des Empowerment in der Sozialen Arbeit. Die vielfältigen Methoden, die in der Empowermentpraxis zum Einsatz kommen, werden ausführlich dargestellt: Unterstü

In [8]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
import torch
# 清理缓存
torch.cuda.empty_cache()

model = SentenceTransformer("models/mpnet-base")
train_dataset = Dataset.from_dict({
    "anchor": [x["sentence1"] for x in x_train],
    "positive": [x["sentence2"] for x in x_train],
})
loss = losses.MultipleNegativesRankingLoss(model)
args = SentenceTransformerTrainingArguments(
    output_dir="models/out",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=20000,
    warmup_steps=100,
    save_total_limit=2,
)
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
    args=args,
)

trainer.train()
if not os.path.exists('models/mpnet-base'):
    os.makedirs('models/mpnet-base')
model.save('models/mpnet-base')

Step,Training Loss
100,2.079400
200,2.079400
300,2.079400
400,2.079400
500,2.079400
600,2.079500
700,2.079400
800,2.079400
900,2.079400
1000,2.079400


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 11.75 GiB of which 137.44 MiB is free. Process 10247 has 16.33 MiB memory in use. Including non-PyTorch memory, this process has 11.21 GiB memory in use. Of the allocated memory 9.86 GiB is allocated by PyTorch, and 1.22 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

model = SentenceTransformer("all-MiniLM-L6-v2")
train_dataset = Dataset.from_dict({
    "anchor": [x["sentence1"] for x in x_train],
    "positive": [x["sentence2"] for x in x_train],
})
loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
if not os.path.exists('all-MiniLM-L6-v2'):
    os.makedirs('all-MiniLM-L6-v2')
model.save('all-MiniLM-L6-v2')

model.safetensors:  58%|#####7    | 52.4M/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss
500,0.913800
1000,0.795400
1500,0.749100
2000,0.709900
2500,0.699000
3000,0.671500
3500,0.668400
4000,0.623100
4500,0.616200
5000,0.616500


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [8]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
train_dataset = Dataset.from_dict({
    "anchor": [x["sentence1"] for x in x_train],
    "positive": [x["sentence2"] for x in x_train],
})
loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
save_path="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
if not os.path.exists(save_path):
    os.makedirs(save_path)
model.save(save_path)

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf-mirror.com/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/eaa086f0ffee582aeb45b36e34cdd1fe2d6de2bef61f8a559a1bbc9bd955917b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1738045763&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODA0NTc2M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvcGFyYXBocmFzZS1tdWx0aWxpbmd1YWwtTWluaUxNLUwxMi12Mi9lYWEwODZmMGZmZWU1ODJhZWI0NWIzNmUzNGNkZDFmZTJkNmRlMmJlZjYxZjhhNTU5YTFiYmM5YmQ5NTU5MTdiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=SoatNHSii88motbZ9AEsMBBZAlfvM-waEJKe2-6LL%7E1yWyECj2mlpzB2PO4ufGA92eNnWtz1BxaGfc3oL16AUJPUk2RXcTN5FICJc2dACHSGU-RYv6wEllyc8x-dm3Z4xA-vCbuJDJJNEXc1gQ%7EfVW8ceK3NXSM12XPLYK2BKTgyYdgSnkL%7EtwO3Ywqvo9X40ZAMvTjhv0UZlBxeH6xiQgdF4HXQY2GDO5KEduUUpfwRGOt9DTKtBL-kxLgkdShRsdBWs5jiMURijeVDfXi5qd6zLGw2kvUdtfmwDJxa

model.safetensors:   2%|2         | 10.5M/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss
500,0.663200
1000,0.587400
1500,0.580700
2000,0.607000
2500,0.562000
3000,0.545000
3500,0.554100
4000,0.517100
4500,0.511200
5000,0.514600


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [31]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
train_dataset = Dataset.from_dict({
    "anchor": [x["sentence1"] for x in x_train],
    "positive": [x["sentence2"] for x in x_train],
})
loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
save_path="sentence-transformers/all-mpnet-base-v2"
if not os.path.exists(save_path):
    os.makedirs(save_path)
model.save(save_path)

model.safetensors:  57%|#####7    | 252M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss
500,0.876200
1000,0.764000
1500,0.715400
2000,0.695800
2500,0.672800
3000,0.660000
3500,0.653300
4000,0.597800
4500,0.592300
5000,0.576700


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [4]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")
train_dataset = Dataset.from_dict({
    "anchor": [x["sentence1"] for x in x_train],
    "positive": [x["sentence2"] for x in x_train],
})
loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
save_path="sentence-transformers/paraphrase-MiniLM-L6-v2"
if not os.path.exists(save_path):
    os.makedirs(save_path)
model.save(save_path)

model.safetensors:  46%|####6     | 41.9M/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss
500,1.142800
1000,0.999200
1500,0.965500
2000,0.886900
2500,0.859700
3000,0.845600
3500,0.817900
4000,0.781200
4500,0.751500
5000,0.720400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [5]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from datasets import Dataset
import os
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

model = SentenceTransformer("sentence-transformers/all-distilroberta-v1")
train_dataset = Dataset.from_dict({
    "anchor": [x["sentence1"] for x in x_train],
    "positive": [x["sentence2"] for x in x_train],
})
loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
save_path="sentence-transformers/all-distilroberta-v1"
if not os.path.exists(save_path):
    os.makedirs(save_path)
model.save(save_path)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss
500,0.866200
1000,0.730500
1500,0.709700
2000,0.633900
2500,0.610400
3000,0.644400
3500,0.578200
4000,0.575900
4500,0.558200
5000,0.522700


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [4]:
import os
import json
import random

root='/media/jh/新加卷/2024_12_04/llms4subjects-main/shared-task-datasets/TIBKAT/'
all_core='tib-core-subjects'
def process_data(root,dev,en_de,datas):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'subject' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])!=str:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])!=str:
            item['abstract']=' '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        item['dcterms:subject']=tmp['dcterms:subject']
        item['subject']=tmp['subject']
        datas.append(item)
    for dirs in os.listdir(root+f'{all_core}/data/{dev}'):
        for file in os.listdir(root+f'{all_core}/data/{dev}/{dirs}/{en_de}'):
            get_train_data(root+f'{all_core}/data/{dev}/{dirs}/{en_de}/{file}')
     
train_datas = []
en_de='en'
dev='train'
process_data(root,dev,en_de,train_datas)
en_de='de'
dev='train'
process_data(root,dev,en_de,train_datas)
random.shuffle(train_datas)

# dev_datas=[]
# en_de='en'
# dev='dev'
# process_data(root,dev,en_de,dev_datas)
# en_de='de'
# dev='dev'
# process_data(root,dev,en_de,dev_datas)
# random.shuffle(dev_datas)

import json
root='/media/jh/新加卷'
file_path=f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())

print(data[0])
gnd_id2name={}
for i in data:
    gnd_id2name[i['Code']]=i['Name']
    for j in i['Alternate Name']:
        gnd_id2name[i['Code']]=gnd_id2name[i['Code']]+' '+j
print(i['Code'],gnd_id2name[i['Code']])


import json

print('len(train_datas)=',len(train_datas))
# print('len(dev_datas)=',len(dev_datas))
datas=train_datas
print('len(datas)=',len(datas))
# datas数据样本随机打乱
import random
print(datas[0])
random.shuffle(datas)
print(datas[0])


from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch


with open('./tmp/core_labels_gte.pkl','rb') as f:
    y_t=pickle.load(f)

model_name_or_path='models/gte_gnd' #"gte-multilingual-base"
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)

def process_data(datas,result):
    batch_size=768
    texts=[datas[i:i+batch_size] for i in range(0, len(datas), batch_size)]
    if len(texts)*batch_size<len(datas):
        texts.append([datas[len(texts)*batch_size:]])
    for block in texts:
        text=[t['title']+'.'+t['abstract'] for t in block]
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
        with torch.no_grad():
            similarities=torch.tensor(similarities).to('cuda')
            # print(similarities.shape) 上次保存的前500个
            top_k = torch.topk(similarities, k=20, dim=0).indices.T.cpu().tolist() # 取出相似度最高的20个
            #similarities中的元素取反
            similarities=-similarities
            top_k_last = torch.topk(similarities, k=20, dim=0).indices.T.cpu().tolist() # 取出相似度最低的20个
        for i,t in enumerate(block):
            pos=[]
            for gnd in t['dcterms:subject']:
                if type(gnd)==dict:
                    gnd_id=gnd['@id']
                else:
                    continue
                if gnd_id in gnd_id2name:
                    pos.append(gnd_id2name[gnd_id])
            for j in top_k[i]:
                pos.append(gnd_id2name[y_t[j]['Code']])
            item1={}
            item1['pos']='.'.join(pos)
            item1['neg']='. '.join([gnd_id2name[y_t[j]['Code']] for j in top_k_last[i]])
            item1['text']=t['title']+'.'+t['abstract']
            result.append(item1)

# 打印运行时间
import time
start_time = time.time()
x_train=[]
process_data(datas,x_train) 
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

{'Code': 'gnd:4003694-7', 'Classification Number': '00', 'Classification Name': 'Unspezifische Allgemeinwörter', 'Name': 'Ausbreitung', 'Alternate Name': [], 'Related Subjects': [], 'Source': 'Du.'}
gnd:1163295434 Airfix Marke
len(train_datas)= 41902
len(datas)= 41902
{'title': 'Introduction to Mathematical Analysis', 'abstract': 'The book begins at an undergraduate student level, assuming only basic knowledge of calculus in one variable. It rigorously treats topics such as multivariable differential calculus, the Lebesgue integral, vector calculus and differential equations. After having created a solid foundation of topology and linear algebra, the text later expands into more advanced topics such as complex analysis, differential forms, calculus of variations, differential geometry and even functional analysis. Overall, this text provides a unique and well-rounded introduction to the highly developed and multi-faceted subject of mathematical analysis as understood by mathematicians 

In [1]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from datasets import Dataset
import os
import torch
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
import pickle

with open('./tmp/gnd_all_train.pkl','rb') as f:
    x_train = pickle.load(f)

model_name_or_path='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
model = SentenceTransformer(model_name_or_path, trust_remote_code=True)

# # 冻结前10层参数
# for name, param in model.named_parameters():
#     if 'layer.11' not in name and 'layer.10' not in name:
#         param.requires_grad = False

args = SentenceTransformerTrainingArguments(
    output_dir='./out',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # 减小批量大小
    per_device_eval_batch_size=8,  # 减小批量大小
    warmup_ratio=0.1,
    fp16=True,  # 混合精度训练
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    eval_strategy="steps",
    eval_steps=1000,
    save_total_limit=2,
    logging_steps=1000,
    # gradient_accumulation_steps=2,  # 梯度累积
)

train_dataset = Dataset.from_dict({
    "anchor": [t['text'] for t in x_train],
    "positive": [t['pos'] for t in x_train],
    "negative": [t['neg'] for t in x_train],
})

train_dataset = train_dataset.train_test_split(test_size=0.1)

loss = losses.TripletLoss(model=model)

trainer = SentenceTransformerTrainer(
    # args=args,
    model=model,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['test'],
    loss=loss,
)

trainer.train()
model.save('models/mpnet_gnd_triplet') # 运行174m


2025-01-18 21:20:46.848702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737206446.863584   65100 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737206446.867164   65100 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 21:20:46.882469: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended,

Step,Training Loss
500,0.043200
1000,0.000000
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.122100
3500,2.971200
4000,4.999600
4500,4.999600
5000,5.001900


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [3]:
import torch
# 清除GPU缓存
torch.cuda.empty_cache()

In [ ]:
import torch
# 清除GPU缓存
torch.cuda.empty_cache()

import os
import json
import random

root='/media/jh/新加卷/2024_12_04/llms4subjects-main/shared-task-datasets/TIBKAT/'
all_core='all-subjects'
def process_data(root,dev,en_de,datas):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'subject' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])!=str:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])!=str:
            item['abstract']=' '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        item['dcterms:subject']=tmp['dcterms:subject']
        item['subject']=tmp['subject']
        datas.append(item)
    for dirs in os.listdir(root+f'{all_core}/data/{dev}'):
        for file in os.listdir(root+f'{all_core}/data/{dev}/{dirs}/{en_de}'):
            get_train_data(root+f'{all_core}/data/{dev}/{dirs}/{en_de}/{file}')
     
train_datas = []
en_de='en'
dev='train'
process_data(root,dev,en_de,train_datas)
en_de='de'
dev='train'
process_data(root,dev,en_de,train_datas)
random.shuffle(train_datas)

# dev_datas=[]
# en_de='en'
# dev='dev'
# process_data(root,dev,en_de,dev_datas)
# en_de='de'
# dev='dev'
# process_data(root,dev,en_de,dev_datas)
# random.shuffle(dev_datas)

import json
root='/media/jh/新加卷'
file_path=f'{root}/2024_12_04/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json' #GND-Subjects-tib-core.json'
with open(file_path,'r') as f:
    data=json.loads(f.read())

print(data[0])
gnd_id2name={}
for i in data:
    gnd_id2name[i['Code']]=i['Name']
    for j in i['Alternate Name']:
        gnd_id2name[i['Code']]=gnd_id2name[i['Code']]+' '+j
print(i['Code'],gnd_id2name[i['Code']])


import json

print('len(train_datas)=',len(train_datas))
# print('len(dev_datas)=',len(dev_datas))
datas=train_datas
print('len(datas)=',len(datas))
# datas数据样本随机打乱
import random
print(datas[0])
random.shuffle(datas)
print(datas[0])


from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch


with open('./tmp/core_labels_mpnet.pkl','rb') as f:
    y_t=pickle.load(f)

model_name_or_path='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'#"gte-multilingual-base"
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)

def process_data(datas,result):
    batch_size=768
    texts=[datas[i:i+batch_size] for i in range(0, len(datas), batch_size)]
    if len(texts)*batch_size<len(datas):
        texts.append([datas[len(texts)*batch_size:]])
    for block in texts:
        text=[t['title']+'.'+t['abstract'] for t in block]
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
        with torch.no_grad():
            similarities=torch.tensor(similarities).to('cuda')
            # print(similarities.shape) 上次保存的前500个
            top_k = torch.topk(similarities, k=20, dim=0).indices.T.cpu().tolist() # 取出相似度最高的20个
            #similarities中的元素取反
            similarities=-similarities
            top_k_last = torch.topk(similarities, k=20, dim=0).indices.T.cpu().tolist() # 取出相似度最低的20个
        for i,t in enumerate(block):
            pos=[]
            for gnd in t['dcterms:subject']:
                if type(gnd)==dict:
                    gnd_id=gnd['@id']
                else:
                    continue
                if gnd_id in gnd_id2name:
                    pos.append(gnd_id2name[gnd_id])
            for j in top_k[i]:
                pos.append(gnd_id2name[y_t[j]['Code']])
            item1={}
            item1['pos']='.'.join(pos)
            item1['neg']='. '.join([gnd_id2name[y_t[j]['Code']] for j in top_k_last[i]])
            item1['text']=t['title']+'.'+t['abstract']
            result.append(item1)

# 打印运行时间
import time
start_time = time.time()
x_train=[]
process_data(datas,x_train) 
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')
# 保存训练数据
with open('./tmp/gnd_all_train.pkl','wb') as f:
    pickle.dump(x_train,f)
import torch
# 清除GPU缓存
torch.cuda.empty_cache()

{'Code': 'gnd:4003694-7', 'Classification Number': '00', 'Classification Name': 'Unspezifische Allgemeinwörter', 'Name': 'Ausbreitung', 'Alternate Name': [], 'Related Subjects': [], 'Source': 'Du.'}
gnd:1163295434 Airfix Marke
len(train_datas)= 81937
len(datas)= 81937
{'title': 'Evolution des Fahrrads', 'abstract': 'Das Fahrrad ist das langlebigste und weitest verbreite Fahrzeug der Welt, von dem in seiner zweihundertjährigen Geschichte mehr als eine Milliarde Exemplare gebaut wurden. Dennoch wurde seine Technikgeschichte bisher vernachlässigt, obwohl es mit seiner Idee des Leichtbaus anstelle der tonnenschweren Eisenbahn und der Abkehr vom Pferd neue Konstruktionen, Materialien und Fertigungstechniken ins Leben rief, welche die Basis für Motorrad-, Automobil- und Flugzeugbau bildeten. Erst seit dreißig Jahren haben eine internationale Konferenz und die mikrohistorische Feinarbeit der nationalen Vintage-Vereine quellenbasierte Erkenntnisse gesammelt, die einen neuen Blick auf die Fahrr

In [14]:
import torch
# 清除GPU缓存
torch.cuda.empty_cache()

In [1]:

from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers

import os
import torch
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"

model_name_or_path = '/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/' #"gte-multilingual-base"
model = SentenceTransformer(model_name_or_path)
# 冻结部分参数: 训练过程中出现loss为0的情况。可能是模型过于复杂。这里我们冻结部分参数，只训练最后一层。
# for name, param in model.named_parameters():
#     if 'encoder' in name:
#         param.requires_grad = False
#     if 'layer.11' in name or 'layer.10' in name:
#         param.requires_grad = True

# from datasets import Dataset
# import pickle
# with open('./tmp/gnd_all_train.pkl','rb') as f:
#     x_train = pickle.load(f)
# train_dataset = Dataset.from_dict({
#     "snentence1": [t['text'] for t in x_train]*2,
#     "sentence2": [t['pos'] for t in x_train]+[t['neg'] for t in x_train],
#     "label": [1]*len(x_train)+[0]*len(x_train),
# })
from datasets import Dataset
import pickle
with open('./tmp/gnd_all_train.pkl','rb') as f:
    x_train = pickle.load(f)
data_tmp=[]
for t in x_train:
    data_tmp.append({'snentence1':t['text'],'sentence2':t['pos'],'label':1})
    data_tmp.append({'snentence1':t['text'],'sentence2':t['neg'],'label':0})
# 打乱数据集
import random
random.shuffle(data_tmp)
train_dataset = Dataset.from_dict({
    "snentence1": [t['snentence1'] for t in data_tmp],
    "sentence2": [t['sentence2'] for t in data_tmp],
    "label": [t['label'] for t in data_tmp],
})

# 打乱数据集
train_dataset = train_dataset.shuffle()
train_dataset = train_dataset.train_test_split(test_size=0.1)

loss = losses.ContrastiveLoss(model)

trainer = SentenceTransformerTrainer(
    # args=args,
    model=model,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['test'],
    loss=loss,
)

trainer.train()
model_out_dir = 'models/mpnet_gte_gnd'
if not os.path.exists(model_out_dir):
    os.makedirs(model_out_dir)
model.save(model_out_dir)


2025-01-18 20:51:05.085841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737204665.102990   63283 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737204665.108285   63283 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-18 20:51:05.127097: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended,

Step,Training Loss
500,0.001000
1000,0.000200
1500,0.000000
2000,0.000100
2500,0.000000


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

KeyboardInterrupt: 

In [3]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

0.auto_model.embeddings.word_embeddings.weight True
0.auto_model.embeddings.position_embeddings.weight True
0.auto_model.embeddings.token_type_embeddings.weight True
0.auto_model.embeddings.LayerNorm.weight True
0.auto_model.embeddings.LayerNorm.bias True
0.auto_model.encoder.layer.0.attention.self.query.weight True
0.auto_model.encoder.layer.0.attention.self.query.bias True
0.auto_model.encoder.layer.0.attention.self.key.weight True
0.auto_model.encoder.layer.0.attention.self.key.bias True
0.auto_model.encoder.layer.0.attention.self.value.weight True
0.auto_model.encoder.layer.0.attention.self.value.bias True
0.auto_model.encoder.layer.0.attention.output.dense.weight True
0.auto_model.encoder.layer.0.attention.output.dense.bias True
0.auto_model.encoder.layer.0.attention.output.LayerNorm.weight True
0.auto_model.encoder.layer.0.attention.output.LayerNorm.bias True
0.auto_model.encoder.layer.0.intermediate.dense.weight True
0.auto_model.encoder.layer.0.intermediate.dense.bias True
0.au

In [3]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses, SentenceTransformerTrainingArguments
from datasets import Dataset
from sentence_transformers.training_args import BatchSamplers
import os
import torch
os.environ["WANDB_API_KEY"] = "4ac3093a84a825c3f9d1ebfeea07254f6412721e"
os.environ["WANDB_MODE"] = "offline"
anchor = [x['sentence1'] for x in x_train]
positive = [x['sentence2'] for x in x_train]
# model_id='sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model_name_or_path="gte-multilingual-base"
model = SentenceTransformer(model_name_or_path, trust_remote_code=True)
model=model.to('cuda')
# 冻结前10层参数
for name, param in model.named_parameters():
    if 'layer.11' not in name:
        param.requires_grad = False
train_dataset = Dataset.from_dict({
    "anchor": anchor,
    "positive": positive,
})
out_dir='/media/jh/新加卷/2024_12_04/gnd_models'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
loss = losses.MultipleNegativesRankingLoss(model)
# 5. (Optional) Specify training arguments
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=out_dir, #"models/mpnet-base-all-nli-triplet",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=160,#16,
    per_device_eval_batch_size=160,#16,
    warmup_ratio=0.1,
    fp16=True, # Set to False if GPU can't handle FP16
    bf16=False, # Set to True if GPU supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES, # MultipleNegativesRankingLoss benefits from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=1000,
    # save_strategy="steps",
    # save_steps=500,
    save_total_limit=2,
    logging_steps=1000,
)
trainer = SentenceTransformerTrainer(
    model=model,
    
    train_dataset=train_dataset,
    loss=loss,
)
trainer.train()
# 6. Save the model

# model.save(out_dir) # core 181m, all: 274m

Some weights of the model checkpoint at gte-multilingual-base were not used when initializing NewModel: {'classifier.bias', 'classifier.weight'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss
500,0.513900
1000,0.405700
1500,0.388200
2000,0.376900
2500,0.359000
3000,0.359700
3500,0.344800
4000,0.342900
4500,0.342000
5000,0.336800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=44004, training_loss=0.273337655819857, metrics={'train_runtime': 6348.5474, 'train_samples_per_second': 55.448, 'train_steps_per_second': 6.931, 'total_flos': 0.0, 'train_loss': 0.273337655819857, 'epoch': 3.0})

In [3]:
# 加载检查点
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/media/jh/新加卷/2025_1_16/tmp_trainer/checkpoint-44004/', trust_remote_code=True)
model.save('./models/gte_gnd')